In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import mne
from scipy.signal import detrend
from scipy.stats import rankdata

In [2]:
sampling_freq = 512 #Hz
l_freq = 70 # highpass
h_freq = 200 # lowpass

In [3]:
def preprocess(patient_id, film_clip_num):
    # Load data
    trial_data = scipy.io.loadmat('/scratch/gpfs/mayaar/GravityECoG/derivatives/preprocessing/sub-ny' + patient_id + '/eeg' + film_clip_num + '_manualica_notch_trial_data.mat')['trial']
    
    # Detrend data (remove linear trend)
    detrended_data = detrend(trial_data)
    
    # Bandpass filter data (70-200 Hz)
    filtered_data = mne.filter.filter_data(detrended_data, sampling_freq, l_freq, h_freq, method='iir')
        
    # Extract signal envelope
    n_channels = filtered_data.shape[0]
    ch_types = ['eeg'] * n_channels
    info = mne.create_info(n_channels, ch_types=ch_types, sfreq=sampling_freq)
    raw = mne.io.RawArray(np.copy(filtered_data), info)
    raw_filtered = raw.apply_hilbert(envelope=True)
    non_ranked_data = raw_filtered[:][0]
        
    # Tie rank data
    tie_ranked_data = rankdata(raw_filtered[:][0], axis=1)
        
    return non_ranked_data, tie_ranked_data